In [3]:

import dspy
from dspy.datasets import HotPotQA
import os

# Set up the OpenAI API key and instantiate the GPT-4o model
api_key = os.getenv("OPENAI_API_KEY")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
lm = dspy.LM("openai/gpt-4o-mini", api_key=api_key)
dspy.configure(lm=lm)

def search(query: str) -> list[str]:
    """Retrieves abstracts from Wikipedia."""
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=500).train]
react = dspy.ReAct("question -> answer", tools=[search])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24)
optimized_react = tp.compile(react, trainset=trainset)

2025/01/11 17:12:34 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 3
valset size: 100



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 3 * 2 lm calls in program + (3) lm calls in program-aware proposer = 19 prompt model calls
- Program Evaluation: 25 examples in minibatch * 7 batches + 100 examples in val set * 1 full evals = 275 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectati

2025/01/11 17:12:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/11 17:12:40 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/11 17:12:40 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=3 sets of demonstrations...


Bootstrapping set 1/3
Bootstrapping set 2/3
Bootstrapping set 3/3


 13%|███████████████▎                                                                                                      | 13/100 [01:08<07:38,  5.27s/it]
2025/01/11 17:13:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/01/11 17:13:49 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


...

2025/01/11 17:16:19 INFO dspy.evaluate.evaluate: Average Metric: 11 / 25 (44.0%)
2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 44.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 1'].
2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [48.0, 44.0, 60.0, 64.0, 48.0, 44.0]
2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [24.0]
2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 24.0
2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: ===========================


2025/01/11 17:16:19 INFO dspy.teleprompt.mipro_optimizer_v2: == Minibatch Trial 7 / 7 ==



Average Metric: 16.00 / 25 (64.0%): 100%|███████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.73it/s]

2025/01/11 17:16:29 INFO dspy.evaluate.evaluate: Average Metric: 16 / 25 (64.0%)
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 64.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 2'].
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [48.0, 44.0, 60.0, 64.0, 48.0, 44.0, 64.0]
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [24.0]
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 24.0
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: ===========================


2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 1 =====
2025/01/11 17:16:29 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 64.0) from minibatch trials...



Average Metric: 51.00 / 100 (51.0%): 100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.15it/s]

2025/01/11 17:16:35 INFO dspy.evaluate.evaluate: Average Metric: 51 / 100 (51.0%)
2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 51.0
2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [24.0, 51.0]
2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 51.0
2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/11 17:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 51.0!


In [5]:
dspy.inspect_history()





[2025-01-11T17:16:35.732802]

System message:

Your input fields are:
1. `question` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Imagine you are a contestant on a high-stakes trivia game show where your knowledge of various topics will determine your success. You will be presented with a question, and your task is to provide an accurate answer based on your reasoning and the information you gather. Use the following fields: `question` to understand what is being asked, and `trajectory` to outline your thought process. Your goal is to interleave your reasoning with tool usage, specifically using a search tool to find relevant i